In [40]:
### from traitlets.config.manager import BaseJSONConfigManager
path = "/usr/local/etc/jupyter/nbconfig"
cm = BaseJSONConfigManager(config_dir=path)
cm.update('livereveal', {
    'theme': 'beige',
    'transition': None, 
    'start_slideshow_at': 'selected',
    'scroll': True,
})

{'scroll': True, 'start_slideshow_at': 'selected', 'theme': 'beige'}

# Docker for Python Development

## How to live 24/7 with Docker builds





## About me:

* Miguel Barrientos
* Developer at Piksel
* [github.com/mbarrientos](https://github.com/mbarrientos)


In this slides...

1. Docker overview
  * docker
  * docker-compose
2. Why is Docker useful at Python apps?
3. Some examples
4. A bunch of tips
5. Some useful images

# Docker overview

Docker architecture
![alt text](images/docker-architecture.svg)

```bash
$ docker run ubuntu:16.04 echo "Hello World"
```

# Creating an image

A very simple Django project

![alt text](images/project_tree.png)

file: Dockerfile
```
FROM python:3.6

RUN apt-get update && \
    apt-get install -y \
        python-psycopg2

COPY . /usr/src/app
WORKDIR /usr/src/app

RUN pip install --no-cache-dir -r requirements.txt

EXPOSE 8000
ENTRYPOINT ["python", "manage.py"]
CMD ["runserver", "0.0.0.0:8000"]
```

Then what?

```bash
$ docker build -t my_app .
```

```bash
$ docker run -it my_app
```

### Quite simple, right?


### Missing something?

## A proper database, maybe? :)

```yaml
## Docker Compose
version: '2'
services:
  app:
    build: .
    image: opensouth
    ports:
      - "8000:8000"
 ```

Equivalent to: 
```bash
docker build . -t opensouth -p 8000:8000 . 
```

```yaml
## Docker Compose
version: '2'
services:
  app:
    build: .
    image: opensouth
    ports:
      - "8000:8000"
    volumes:
      - .:/usr/src/app/
    links:
      - postgres
    environment:
      - DB_DEFAULT_HOST=postgres
      - DB_DEFAULT_PORT=5432
      - DB_DEFAULT_USER=opensouth
      - DB_DEFAULT_PASSWORD=DockerIsGreat
      - DB_DEFAULT_NAME=opensouth

  postgres:
    image: postgres
    environment:
      - POSTGRES_USER=opensouth
      - POSTGRES_PASSWORD=DockerIsGreat
      - POSTGRES_DB=opensouth
 ```

```yaml
## Docker Compose
version: '2'
services:
  app:
    build: .
    image: opensouth
    ports:
      - "80:8000"
    volumes:
      - .:/usr/src/app/
      - ./logs/opensouth:/var/log/opensouth
    links:
      - postgres
    environment:
      - DB_DEFAULT_HOST=${DB_DEFAULT_HOST}
      - DB_DEFAULT_PORT=${DB_DEFAULT_PORT}
      - DB_DEFAULT_USER=${DB_DEFAULT_USER}
      - DB_DEFAULT_PASSWORD=${DB_DEFAULT_PASSWORD}
      - DB_DEFAULT_NAME=${DB_DEFAULT_NAME}
      - DJANGO_SECRET_KEY=${DJANGO_SECRET_KEY}

  postgres:
    image: postgres
    volumes:
      - ./.data/pgdata:${PGDATA}
      - ./logs/postgresql:/var/log/postgresql
    environment:
      - POSTGRES_USER=${DB_DEFAULT_USER}
      - POSTGRES_PASSWORD=${DB_DEFAULT_PASSWORD}
      - POSTGRES_DB=${DB_DEFAULT_NAME}
      - PGDATA=/var/lib/postgresql/data/pgdata
 ```